In [1]:
import os, zipfile
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import nibabel as nib # 用于读取.nii/.nii.gz格式的文件
from scipy import ndimage # 用于图像处理(旋转/缩放)

In [15]:
def read_nifti_file(filepath):
    scan = nib.load(filepath)
    scan = scan.get_fdata() # 从NfTI文件中提取出图像数据并转换为float 最终返回bumpy数组
    return scan

def normaliza(volume):
    # 将CT值裁剪到-1000～400范围 
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    # 归一化
    volume = (volume-min)/(max-min)
    volume = volume.astype('float32')
    return volume

def resize_volume(img):
    desired_depth = 64
    desired_width = 128
    desired_height = 128

    # 初始深宽高
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]

    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    
    # 计算出个维度缩放比例
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height

    img = ndimage.rotate(img, 90, reshape=False) # 纠正图像朝向

    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1) # 对n维数组进行缩放
    return img

def process_scan(path): # 整合上述步骤
    volume = read_nifti_file(path)
    volume = normaliza(volume)
    volume = resize_volume(volume)
    return volume

In [21]:
normal_scan_paths = [ # 正常样本路径
    os.path.join(os.getcwd(), 'CT-0', x)
    for x in os.listdir('CT-0')
    if x.endswith('.nii') or x.endswith('.nii.gz')
]

abnormal_scan_paths = [ # 异常样本路径
    os.path.join(os.getcwd(), 'CT-23', x)
    for x in os.listdir('CT-23')
    if x.endswith('.nii') or x.endswith('.nii.gz')
]

print('CT scans with normal lung tissue:' + str(len(normal_scan_paths)))
print('CT scans with abnormal lung tissue:' + str(len(abnormal_scan_paths)))

CT scans with normal lung tissue:100
CT scans with abnormal lung tissue:100


In [23]:
# 处理后存入numpy数组
abnormal_scans = np.array([process_scan(path) for path in abnormal_scan_paths])
normal_scans = np.array([process_scan(path) for path in normal_scan_paths])

abnormal_labels = np.array([1 for _ in range(len(abnormal_scans))])
normal_labels = np.array([0 for _ in range(len(normal_scans))])

In [26]:
# 划分训练集和验证集
x_train = np.concatenate((abnormal_scans[:70], normal_scans[:70]), axis=0)
y_train = np.concatenate((abnormal_labels[:70], normal_labels[:70]), axis=0)
x_val = np.concatenate((abnormal_scans[70:], normal_scans[70:]), axis=0)
y_val = np.concatenate((abnormal_labels[70:], normal_labels[70:]), axis=0)

print('Number of samples in train and validation are %d and %d.' % (x_train.shape[0], x_val.shape[0]))

Number of samples in train and validation are 140 and 60.


In [74]:
import random
from scipy import ndimage

@tf.function # 转换为Tensorflow图模式 提高性能
def rotate(volume):
    '''不同程度上进行旋转'''
    def scipy_rotate(volume):
        angles = [-20, -10, -5, 5, 10, 20] # 可选旋转角度
        angle = random.choice(angles) # 随机选择一个角度

        volume = ndimage.rotate(volume, angle, reshape=False) # 执行旋转
        # 确保所有值都在0-1间
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    # 在tensorflow静态计算图内部执行用python/numpy/scipy编写的函数
    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32) # func:要调用的python函数 inp:一个list 包含要传递给func的参数 Tonc:指定python函数func返回值的tensorflow类型
    augmented_volume.set_shape(volume.shape) # 手动设置张量的形状信息
    return augmented_volume

# 为何增加通道数? 
# 2D CNN_input=(batch_size, height, width, channels)
# 3D CNN_input=(batch_size, height, width, depth, channels)
# 在axis=3处增加channel维即可
def train_preprocessing(volume, label):
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

def validation_preprocessing(volume, label):
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

In [76]:
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))

batch_size = 2

train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

In [ ]:
import matplotlib.pyplot as plt

data = train_dataset.take(1)
images, labels = list(data)[0]
images = images.numpy()
image = images[0]
print('Dimension of the CT scan is:', image.shape)
plt.imshow(np.squeeze(image[:, :, 30]), cmap='grey')

In [ ]:
def plot_slices(num_rows, num_columns, width, height, data):
    data = np.rot90(np.array(data))
    data = np.transpose(data)
    data = np.reshape(data, (num_rows, num_columns, width, height))
    rows_data, columns_data = data.shape[0], data.shape[1]
    heights = [slc[0].shape[0] for slc in data]
    widths = [slc.shape[1] for slc in data[0]]
    fig_width = 12.0
    fig_height = fig_width * sum(heights) / sum(widths)
    f, axarr = plt.subplots(
        rows_data, 
        columns_data,
        figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios': heights}
    )
    for i in range (rows_data):
        for j in range(columns_data):
            axarr[i, j].imshow(data[i][j], cmap='gray')
            axarr[i, j].axis('off')
    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    plt.show()

plot_slices(4, 10, 128, 128, image[:, :, :40])

In [78]:
def get_model(width=128, height=128, depth=64):
    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation='relu')(inputs)
    x = layers.MaxPooling3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs, name='3dcnn')
    return model

model = get_model(width=128, height=128, depth=64)
model.summary()

Model: "3dcnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 64,   │             0 │
│                                 │ 1)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (None, 126, 126, 62,   │         1,792 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_4 (MaxPooling3D)  │ (None, 63, 63, 31, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 63, 63, 31, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_5 (Conv3D)               │ (None, 61, 61, 29, 64) │       110,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_5 (MaxPooling3D)  │ (None, 30, 30, 14, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 30, 30, 14, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_6 (Conv3D)               │ (None, 28, 28, 12,     │       221,312 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_6 (MaxPooling3D)  │ (None, 14, 14, 6, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 14, 14, 6, 128) │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_7 (Conv3D)               │ (None, 12, 12, 4, 256) │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_7 (MaxPooling3D)  │ (None, 6, 6, 2, 256)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 6, 6, 2, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling3d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling3D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,352,897 (5.16 MB)

 Trainable params: 1,351,873 (5.16 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [80]:
initial_learning_rate = 1e-4
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=30, decay_rate=0.96, staircase=True
)

model.compile(
    loss = 'binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=['acc']
)

checkpoint_cb = keras.callbacks.ModelCheckpoint(
    '3d_image_classification.h5', save_best_only=True
)

# 回调函数 自动提前停止训练 防止过拟合
# monitor:监控指标 patience:验证集准确率在连续xx个epoch都没有变化得比历史最佳值更好 则停止训练
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_acc', patience=15)

epochs = 100
model.fit(
    train_dataset, 
    validation_data = validation_dataset,
    epochs = epochs,
    shuffle = True,
    verbose = 2,
    callbacks = [checkpoint_cb, early_stopping_cb]
)


Epoch 1/100


70/70 - 362s - 5s/step - acc: 0.5000 - loss: 0.7125 - val_acc: 0.5000 - val_loss: 1.1491
Epoch 2/100
70/70 - 369s - 5s/step - acc: 0.6000 - loss: 0.6643 - val_acc: 0.5000 - val_loss: 1.8033
Epoch 3/100
70/70 - 359s - 5s/step - acc: 0.6429 - loss: 0.6527 - val_acc: 0.5000 - val_loss: 2.0689
Epoch 4/100
70/70 - 354s - 5s/step - acc: 0.6214 - loss: 0.6637 - val_acc: 0.5000 - val_loss: 1.2622
Epoch 5/100
70/70 - 353s - 5s/step - acc: 0.7214 - loss: 0.6151 - val_acc: 0.5000 - val_loss: 1.6500
Epoch 6/100


70/70 - 360s - 5s/step - acc: 0.5857 - loss: 0.6609 - val_acc: 0.5000 - val_loss: 0.7535
Epoch 7/100
70/70 - 341s - 5s/step - acc: 0.6929 - loss: 0.6028 - val_acc: 0.5500 - val_loss: 0.8240
Epoch 8/100
70/70 - 346s - 5s/step - acc: 0.6286 - loss: 0.6342 - val_acc: 0.5833 - val_loss: 0.8512
Epoch 9/100


70/70 - 351s - 5s/step - acc: 0.6357 - loss: 0.6211 - val_acc: 0.6667 - val_loss: 0.6651
Epoch 10/100


70/70 - 372s - 5s/step - acc: 0.7071 - loss: 0.5902 - val_acc: 0.7167 - val_loss: 0.5660
Epoch 11/100


70/70 - 369s - 5s/step - acc: 0.6857 - loss: 0.5892 - val_acc: 0.6667 - val_loss: 0.5454
Epoch 12/100
70/70 - 367s - 5s/step - acc: 0.6429 - loss: 0.6253 - val_acc: 0.7000 - val_loss: 0.5586
Epoch 13/100
70/70 - 371s - 5s/step - acc: 0.6786 - loss: 0.5886 - val_acc: 0.7167 - val_loss: 0.5603
Epoch 14/100
70/70 - 388s - 6s/step - acc: 0.7429 - loss: 0.5606 - val_acc: 0.6833 - val_loss: 0.5704
Epoch 15/100
70/70 - 379s - 5s/step - acc: 0.7214 - loss: 0.5564 - val_acc: 0.6500 - val_loss: 0.5709
Epoch 16/100
70/70 - 359s - 5s/step - acc: 0.7071 - loss: 0.5810 - val_acc: 0.6667 - val_loss: 0.5719
Epoch 17/100


70/70 - 371s - 5s/step - acc: 0.7571 - loss: 0.5410 - val_acc: 0.7000 - val_loss: 0.5398
Epoch 18/100
70/70 - 394s - 6s/step - acc: 0.6857 - loss: 0.5458 - val_acc: 0.7333 - val_loss: 0.5437
Epoch 19/100
70/70 - 379s - 5s/step - acc: 0.7643 - loss: 0.5429 - val_acc: 0.7167 - val_loss: 0.5593
Epoch 20/100
70/70 - 397s - 6s/step - acc: 0.7857 - loss: 0.5174 - val_acc: 0.7167 - val_loss: 0.5406
Epoch 21/100
70/70 - 361s - 5s/step - acc: 0.7071 - loss: 0.5221 - val_acc: 0.6833 - val_loss: 0.5574
Epoch 22/100
70/70 - 358s - 5s/step - acc: 0.6857 - loss: 0.5597 - val_acc: 0.6833 - val_loss: 0.5721
Epoch 23/100
70/70 - 364s - 5s/step - acc: 0.7429 - loss: 0.5581 - val_acc: 0.7000 - val_loss: 0.5722
Epoch 24/100
70/70 - 353s - 5s/step - acc: 0.7357 - loss: 0.5439 - val_acc: 0.6667 - val_loss: 0.6387
Epoch 25/100
70/70 - 338s - 5s/step - acc: 0.7786 - loss: 0.5070 - val_acc: 0.6833 - val_loss: 0.5582
Epoch 26/100
70/70 - 1123s - 16s/step - acc: 0.7929 - loss: 0.4930 - val_acc: 0.6333 - val_loss

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 3))
ax = ax.ravel()

for i, metric in enumerate(['acc', 'loss']):
    ax[i].plot(model.history.history[metric])
    ax[i].plot(model.history.history['val_'+metric])
    ax[i].set_title('Model ()'.format(metric))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(metric)
    ax[i].legend(['train', 'val'])

In [86]:
model.load_weights('3d_image_classification.h5')
prediction = model.predict(np.expand_dims(x_val[0], axis=0))[0] # 增加批量维度 返回一个批量的预测结果 取第一个批次的结果
scores = [1 - prediction[0], prediction[0]]

class_names = ['normal', 'abnormal']
for score, name in zip(scores, class_names):
    print('This model is %.2f percent confident that CT scan is %s' % ((100 * score), name))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
This model is 35.53 percent confident that CT scan is normal
This model is 64.47 percent confident that CT scan is abnormal
